In [31]:
import numpy as np
import scipy.optimize as opt
import pylab as plt
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import math

In [32]:
kB = 0.695 #boltzmans constant in wavenumber units per kelvin (cm^-1/K)
T = 300 #K. Completely random. Can change to whatever
k=0.025 #time step (ns). Can change as desired

In [33]:
#Loop to do many runs 
runs=500
output_runs=np.full((runs,1),0.000)

for x in range(0,runs):

    latticeE=np.random.normal(11000, 200, size=(4,4,4)) #randomly generating disordered latti
    W2=np.full((64,64),0.000)#initializing W2 matrix as all zero, fill in w loop
    P1=np.full((len(W2),1),0.015625) #initial distribution, all elements equal
    lengthInner=len(latticeE[0])
    lengthOuter=len(latticeE)
    lengthDepth=len(latticeE)
    rowCounter = 0
    for i in range (0,len(latticeE)): #This loop is defining the nearest neighbors (periodic BCs)
        for j in range (0,len(latticeE)):
            for k in range(0,len(latticeE)):
                if (j==0 or j==lengthInner-1):
                    neighborWrap = j + ((lengthInner-1) * (1 if j==0 else -1))
                    neighborNonWrap = j + (1 if j==0 else -1)
                    neighborLeft = neighborWrap if (j==0) else neighborNonWrap
                    neighborRight = neighborNonWrap if (j==0) else neighborWrap
                else:
                    neighborLeft = j - 1
                    neighborRight = j + 1
                if (i==0 or i==lengthOuter-1):
                    neighborWrap = i + ((lengthOuter-1) * (1 if i==0 else -1))
                    neighborNonWrap = i + (1 if i==0 else -1)
                    neighborUp = neighborWrap if (i==0) else neighborNonWrap
                    neighborDown = neighborNonWrap if (i==0) else neighborWrap
                else:
                    neighborUp = i - 1
                    neighborDown = i + 1
                if (k==0 or k==lengthDepth-1):
                    neighborWrap=k+(lengthDepth-1)*(1 if k==0 else -1)
                    neighborNonWrap=k+(1 if k==0 else -1)
                    neighborFront=neighborWrap if (k==0) else neighborNonWrap
                    neighborBack=neighborNonWrap if (k==0) else neighborWrap
                iLeft = 16*k+(i * len(latticeE)) + neighborLeft #for n*n*n lattice, change 16 to n^2
                iRight = 16*k+(i * len(latticeE)) + neighborRight
                iUp = 16*k+(neighborUp * len(latticeE)) + j
                iDown = 16*k+(neighborDown * len(latticeE)) + j 
                iFront= 16*k +neighborFront
                iBack=16*k +neighborBack
                #After this we enter the probabilities depending on neighbors to W2
                W2[iLeft][rowCounter] = math.exp(-(latticeE[k][i][neighborLeft]-latticeE[k][i][j])/(kB*T)) if (latticeE[k][i][neighborLeft] > latticeE[k][i][j]) else 1
                W2[iRight][rowCounter] = math.exp(-(latticeE[k][i][neighborRight]-latticeE[k][i][j])/(kB*T)) if (latticeE[k][i][neighborRight] > latticeE[k][i][j]) else 1
                W2[iUp][rowCounter] = math.exp(-(latticeE[k][neighborUp][j]-latticeE[k][i][j])/(kB*T)) if (latticeE[k][neighborUp][j] > latticeE[k][i][j]) else 1
                W2[iDown][rowCounter] = math.exp(-(latticeE[k][neighborDown][j]-latticeE[k][i][j])/(kB*T)) if (latticeE[k][neighborDown][j] > latticeE[k][i][j]) else 1
                W2[iFront][rowCounter] = math.exp(-(latticeE[neighborFront][i][j]-latticeE[k][i][j])/(kB*T)) if (latticeE[neighborFront][i][j] > latticeE[k][i][j]) else 1
                W2[iBack][rowCounter] = math.exp(-(latticeE[neighborBack][i][j]-latticeE[k][i][j])/(kB*T)) if (latticeE[neighborBack][i][j] > latticeE[k][i][j]) else 1
                rowCounter += 1

    W2=k*W2 #scaling by k
    for i in range (0, len(W2)): #diagonal elements once the transitions are calculated and scaled by k
        for j in range(0, len(W2)):
            if (i == j):
                W2[i][j] = 1-np.sum(W2, axis=0)[i] #to ensure all rows sum to 1

        
    Boltz=np.full((len(W2),1),0.000) #initializing boltzman dist as zero, then 
    LatticeList= latticeE.ravel()
    Plist=np.full((len(W2),1),0.000) #empty list for boltzman, will divide by partition function for Boltz

    for i in range (0,len(LatticeList)):
        p_i = math.exp(-(LatticeList[i])/(kB*T))
        Plist[i]=p_i

    for i in range (0,len(LatticeList)):
        Q= np.sum(Plist, axis=0) #partition function

    for i in range (0,len(LatticeList)):
        Boltz[i]= Plist[i]/Q  

    arr = P1
    boundLimit = 0.001 #how close each element of Pn must be to Boltz values 
    notWithinBounds = True
    ctr = 0
    while notWithinBounds:
        arrInBounds = True
        for i in range (0, len(arr)):
            if (abs(arr[i]-Boltz[i]) > boundLimit):
                arrInBounds = False
                break
        if not(arrInBounds):
            arr = W2 @ arr
            ctr = ctr + 1
        else:
            notWithinBounds = False

    print(ctr)
    output_runs[x]=ctr




<ipython-input-33-1f333d24bfaa>:85: RuntimeWarning: overflow encountered in matmul
  arr = W2 @ arr
<ipython-input-33-1f333d24bfaa>:85: RuntimeWarning: invalid value encountered in matmul
  arr = W2 @ arr


239
236
237
239
237
233
234
240
233
235
235
239
235
240
242
241
237
237
236
235
236
235
236
234
234
235
243
236
235
236
234
234
235
234
236
233
238
233
236
240
241
235
235
240
237
238
234
237
235
238
238
234
241
235
235
238
237
242
235
241
237
240
234
237
233
236
242
232
237
236
234
234
239
237
237
237
234
237
235
234
239
241
238
236
239
238
237
235
234
237
236
237
236
236
236
236
240
235
241
235
238
234
236
238
233
238
235
235
238
236
236
233
236
236
240
237
245
237
240
233
235
238
237
236
235
238
235
234
233
233
234
238
235
237
236
238
236
236
235
239
237
235
233
238
236
236
236
237
233
236
240
236
240
238
237
235
237
241
237
235
234
235
235
237
238
237
239
235
233
238
238
236
239
235
237
234
237
236
239
239
233
234
235
234
235
235
236
236
237
236
237
239
237
238
244
237
243
233
231
238
237
237
236
236
237
234
232
237
233
236
235
235
243
236
239
242
237
238
236
235
238
240
239
241
234
234
238
238
239
241
237
240
235
239
239
237
238
233
237
232
235
236
238
241
239
233
236
242
231
234


In [29]:
time_eq=k*output_runs #converting runs to time

4


In [ ]:
n, bins, patches = plt.hist(x=time_eq, bins=70, color='#0504aa',
                            alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.85)
plt.xlabel('Time to Equillibrium')
plt.ylabel('Frequency')
plt.title('Distribution of runs(70 bins): energetic disorder of 500')
maxfreq = n.max()
# Set a clean upper y-axis limit.
plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)
print(bins)

In [ ]:
import scipy.stats as stats
from scipy.stats import norm, kurtosis, skew
kurtosis(time_eq, fisher=False)
skew(time_eq)
min(time_eq)
max(time_eq)